In [40]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics

In [52]:
df_train = pd.read_csv("./new/train_data.csv")
df_test = pd.read_csv("./new/test_data.csv")

In [53]:
df_train = df_train.replace(np.inf, np.nan)
df_train = df_train.fillna(0)

df_test = df_test.replace(np.inf, np.nan)
df_test = df_test.fillna(0)

In [54]:
#Clean useless columns
df_label = df_train.pop("segment")
df_train.drop("ID", axis = 1, inplace=True)

#Validation split
x_train, x_valid, label_train, label_valid = train_test_split(df_train, df_label, test_size=0.2, random_state=4242, stratify = df_label)

test_ids = df_test.pop("ID")

In [55]:
#train final model
rf_model = RandomForestClassifier(n_estimators=300,max_depth=6, max_features=10)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features=10, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [56]:
#Validation
rf_model.fit(x_train, label_train)
print sorted(list(zip(x_train.columns, rf_model.feature_importances_)), key=lambda x:x[1], reverse = True)
rf_pred = rf_model.predict_proba(x_valid)
print metrics.roc_auc_score(label_valid, [x[1] for x in rf_pred])

0.827934123305


In [57]:
rf_model.fit(df_train, df_label)
print sorted(list(zip(df_train.columns, rf_model.feature_importances_)), key=lambda x:x[1], reverse = True)
rf_pred = rf_model.predict_proba(df_test)

[('segment_view_ratio', 0.24460814502429684), ('sum_wt_ratio', 0.22069125501997686), ('generated_positive_ratio', 0.086692454788716386), ('view_within_seg', 0.073916617891573153), ('generated_sports_ratio', 0.058230993096293954), ('generated_positive', 0.04124507467370709), ('generated_sports', 0.037105813892649886), ('segment_wt_median_ratio', 0.036080898661986972), ('titles_mean', 0.031068813420199889), ('super_positive_tod', 0.027526100728673484), ('segment_wt_mean_ratio', 0.023178584828949206), ('titles_max', 0.021402559547713227), ('sum_wt', 0.018159035032001208), ('titles_gt_600', 0.011296997332546481), ('indi_nonsegment_ratio', 0.0094517431154180799), ('indi_segment_ratio', 0.0090250279230479954), ('positive_tod', 0.0088240468546052046), ('cities_count', 0.007502813649843754), ('titles_median', 0.0063056181956187458), ('super_positive_tod_ratio', 0.0045828232865500473), ('dow_count', 0.003839623470648321), ('title_count', 0.0032477072948844847), ('titles_max_perc', 0.00176880870

In [58]:
sub = pd.DataFrame()
sub['ID'] = test_ids
sub['segment'] = [x[1] for x in rf_pred]
sub.to_csv("./subs/rf_1.csv", index=False)   

ValueError: Wrong number of items passed 2, placement implies 1